In [6]:
#
#    Copyright (c) 2025 Tom Keffer <tkeffer@gmail.com>
#
#    See the file LICENSE.txt for your rights.
#
"""
Calculate 1-D bottom line
"""
import os.path
import glob
import warnings
from pathlib import Path

import echopype.qc
import echopype as ep
import numpy as np
import xarray as xr

warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)

# Path to the root data directory:
data_path = Path("~/Data/Western_Flyer/baja2025/ek80/").expanduser()

echodata_zarr_path = data_path / "echodata_zarr"
Sv_zarr_path = data_path / "Sv_zarr"
MVBS_zarr_path = data_path / "MVBS_zarr"

## Focus on a single file
To start, let's focus on a single file: 2025-04-30 at 22:25:42 UTC:

In [2]:
stem = "250430WF-D20250430-T222542"
sv_file_path = Sv_zarr_path / (stem + "_Sv.zarr")

Open it

In [3]:
sv_ds = xr.open_zarr(sv_file_path)
sv_ds

<xarray.Dataset> Size: 399MB
Dimensions:                        (channel: 2, ping_time: 66,
                                    range_sample: 125726, filenames: 1)
Coordinates:
  * channel                        (channel) <U36 288B 'WBT Mini 278014-7 ES3...
  * ping_time                      (ping_time) datetime64[ns] 528B 2025-04-30...
  * range_sample                   (range_sample) int64 1MB 0 1 ... 125725
  * filenames                      (filenames) int64 8B 0
Data variables: (12/25)
    Sv                             (channel, ping_time, range_sample) float64 133MB dask.array<chunksize=(1, 9, 15716), meta=np.ndarray>
    angle_offset_alongship         (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    angle_offset_athwartship       (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    angle_sensitivity_alongship    (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    angle_sensitivity_athwartship  (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    beamwidth_alongship            (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    ...                             ...
    salinity                       float64 8B ...
    sound_absorption               (channel) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    sound_speed                    float64 8B ...
    source_filenames               (filenames) <U92 368B dask.array<chunksize=(1,), meta=np.ndarray>
    temperature                    float64 8B ...
    water_level                    float64 8B ...
Attributes:
    processing_function:          calibrate.compute_Sv
    processing_software_name:     echopype
    processing_software_version:  0.10.1
    processing_time:              2025-11-09T02:27:43Z

Pull out just the `depth` DataArray, then start working with it:

In [198]:
depth = sv_ds['depth']
depth

<xarray.DataArray 'depth' (channel: 2, ping_time: 66, range_sample: 125726)> Size: 133MB
dask.array<open_dataset-depth, shape=(2, 66, 125726), dtype=float64, chunksize=(1, 9, 15716), chunktype=numpy.ndarray>
Coordinates:
  * channel       (channel) <U36 288B 'WBT Mini 278014-7 ES38-18|200-18C_ES' ...
  * ping_time     (ping_time) datetime64[ns] 528B 2025-04-30T22:25:42.353058 ...
  * range_sample  (range_sample) int64 1MB 0 1 2 3 ... 125723 125724 125725
Attributes:
    history:  2025-11-09 02:27:43.528481 +00:00. depth` calculated using: Sv ...

Select the 38 kHz channel, then look at the ping 10 around index 300. From this, range sampling seems to about every 3 cm.

In [199]:
depth.sel(channel='WBT Mini 278014-7 ES38-18|200-18C_ES')[10].isel(range_sample=slice(250,350)).values

array([ 8.45746273,  8.48729259,  8.51712244,  8.54695229,  8.57678214,
        8.60661199,  8.63644184,  8.66627169,  8.69610154,  8.72593139,
        8.75576124,  8.78559109,  8.81542095,  8.8452508 ,  8.87508065,
        8.9049105 ,  8.93474035,  8.9645702 ,  8.99440005,  9.0242299 ,
        9.05405975,  9.0838896 ,  9.11371945,  9.14354931,  9.17337916,
        9.20320901,  9.23303886,  9.26286871,  9.29269856,  9.32252841,
        9.35235826,  9.38218811,  9.41201796,  9.44184782,  9.47167767,
        9.50150752,  9.53133737,  9.56116722,  9.59099707,  9.62082692,
        9.65065677,  9.68048662,  9.71031647,  9.74014632,  9.76997618,
        9.79980603,  9.82963588,  9.85946573,  9.88929558,  9.91912543,
        9.94895528,  9.97878513, 10.00861498, 10.03844483, 10.06827468,
       10.09810454, 10.12793439, 10.15776424, 10.18759409, 10.21742394,
       10.24725379, 10.27708364, 10.30691349, 10.33674334, 10.36657319,
       10.39640305, 10.4262329 , 10.45606275, 10.4858926 , 10.51

In the above, the first sample is at index 250. We find 10 m 52 cells in, so 10 m should be around index 302.

In [206]:
depth.sel(channel='WBT Mini 278014-7 ES38-18|200-18C_ES')[10].isel(range_sample=302).values

array(10.00861498)

If we skip 300 bins from the surface, the algorithm will start looking for the bottom starting at 10 m.
Look for the sea floor, using the 38 kHz channel.

In [207]:
sea_floor = ep.mask.detect_seafloor(sv_ds, 'basic', {
    "var_name": "Sv", "channel": 'WBT Mini 278014-7 ES38-18|200-18C_ES',
    "threshold": -40, "offset_m": 1.0, "bin_skip_from_surface": 300
})
sea_floor

<xarray.DataArray 'bottom_depth' (ping_time: 66)> Size: 528B
dask.array<sub, shape=(66,), dtype=float64, chunksize=(9,), chunktype=numpy.ndarray>
Coordinates:
  * ping_time  (ping_time) datetime64[ns] 528B 2025-04-30T22:25:42.353058 ......
Attributes:
    detector:               basic
    threshold_min:          -40.0
    threshold_max:          -30.0
    offset_m:               0.5
    bin_skip_from_surface:  300
    channel:                WBT Mini 278014-7 ES38-18|200-18C_ES

In [208]:
sea_floor.values

array([131.30389636, 129.03682769, 129.21580679, 129.42461575,
       129.54393515, 129.63342471, 129.75274411, 129.96155307,
       130.20019188, 130.37917098, 130.58797994, 130.88627845,
       130.76695904, 130.94593815, 131.12491725, 131.27406651,
       131.42321576, 131.57236502, 131.78117397, 131.96015308,
       132.19879189, 132.64623965, 132.58657995, 132.76555906,
       132.97436801, 133.18317697, 133.36215607, 133.54113518,
       133.80960384, 133.98858294, 134.2570516 , 134.37637101,
       134.61500981, 134.88347847, 135.18177698, 135.39058594,
       135.59939489, 135.778374  , 135.95735311, 136.19599191,
       136.37497102, 136.55395012, 136.76275908, 137.03122774,
       137.15054714, 137.44884565, 137.50850535, 137.74714416,
       137.86646357, 138.01561282, 138.19459193, 138.34374118,
       138.55255014, 138.70169939, 138.85084865, 138.97016805,
       139.14914716, 139.38778596, 139.59659492, 139.71591432,
       139.89489343, 140.01421283, 140.16336209, 140.28

## Concatenation
Take a look at the next file, which has a start time of 22:26:54, using the same threshold:

In [210]:
stem2 = "250430WF-D20250430-T222654"
sv_file_path2 = Sv_zarr_path / (stem + "_Sv.zarr")

In [211]:
sv_ds2 = xr.open_zarr(sv_file_path2)
sea_floor2 = ep.mask.detect_seafloor(sv_ds2, 'basic', {
    "var_name": "Sv", "channel": 'WBT Mini 278014-7 ES38-18|200-18C_ES',
    "threshold": -40, "offset_m": 1.0, "bin_skip_from_surface": 300
})
sea_floor2

<xarray.DataArray 'bottom_depth' (ping_time: 66)> Size: 528B
dask.array<sub, shape=(66,), dtype=float64, chunksize=(9,), chunktype=numpy.ndarray>
Coordinates:
  * ping_time  (ping_time) datetime64[ns] 528B 2025-04-30T22:26:54.875528 ......
Attributes:
    detector:               basic
    threshold_min:          -40.0
    threshold_max:          -30.0
    offset_m:               1.0
    bin_skip_from_surface:  300
    channel:                WBT Mini 278014-7 ES38-18|200-18C_ES

Now concatenate the two files together.

In [196]:
long_floor = xr.concat([sea_floor, sea_floor2],dim='ping_time')
long_floor

<xarray.DataArray 'bottom_depth' (ping_time: 132)> Size: 1kB
dask.array<concatenate, shape=(132,), dtype=float64, chunksize=(9,), chunktype=numpy.ndarray>
Coordinates:
  * ping_time  (ping_time) datetime64[ns] 1kB 2025-04-30T22:25:42.353058 ... ...
Attributes:
    detector:               basic
    threshold_min:          -40.0
    threshold_max:          -30.0
    offset_m:               0.5
    bin_skip_from_surface:  300
    channel:                WBT Mini 278014-7 ES38-18|200-18C_ES

 ## Process data for 2025-Apr-30
 Collect data for 2025-Apr-30, when we were north of Isla San Marcos.

In [184]:
sv_glob = os.path.join(Sv_zarr_path, "250430WF-D20250430-T*_Sv.zarr")
sv_files = sorted(glob.glob(sv_glob))
print(f"Found {len(sv_files)} files that satisfy the glob pattern")

Found 217 files that satisfy the glob pattern


In [185]:
%%time

sea_floors_list = []
for sv_file in sv_files:
    sv_ds = xr.open_zarr(sv_file)
    sea_floor = ep.mask.detect_seafloor(sv_ds, 'basic', {
        "var_name": "Sv", "channel": 'WBT Mini 278014-7 ES38-18|200-18C_ES',
        "threshold": -40, "offset_m": 1.0, "bin_skip_from_surface": 300
    })
    sea_floors_list.append(sea_floor)
# Concatenate them all together:
sea_floors_da = xr.concat(sea_floors_list, dim='ping_time')
# Subsample to 10 second boundaries:
da_resampled = sea_floors_da.resample(ping_time='10s').interpolate('linear')
da_resampled

CPU times: user 39.1 s, sys: 4.84 s, total: 43.9 s
Wall time: 28.7 s


<xarray.DataArray 'bottom_depth' (ping_time: 1646)> Size: 13kB
dask.array<transpose, shape=(1646,), dtype=float64, chunksize=(1646,), chunktype=numpy.ndarray>
Coordinates:
  * ping_time  (ping_time) datetime64[ns] 13kB 2025-04-30T19:26:00 ... 2025-0...
Attributes:
    detector:               basic
    threshold_min:          -40.0
    threshold_max:          -30.0
    offset_m:               0.5
    bin_skip_from_surface:  300
    channel:                WBT Mini 278014-7 ES38-18|200-18C_ES

## GPS data
Do something similar for the GPS data. The Sv data does not have `Platform` data in it, so we have to start with the converted Zarr data. 

For now, work with one file. We also only interested in the `Platform` data, so read just that.

In [4]:

ed_glob = os.path.join(echodata_zarr_path, "250430WF-D20250430-T221*.zarr")
ed_files = sorted(glob.glob(ed_glob))
print(f"Found {len(ed_files)} files that satisfy the glob pattern")

Found 8 files that satisfy the glob pattern


In [5]:
%%time

latitudes = []
longitudes = []

for ed_file in ed_files:
    # Because the metadata in our data have not been consolidated, we need
    # the option consolidated=False to avoid tons of warnings:
    ed_ds = xr.open_dataset(f"{ed_file}/Platform", engine="zarr", consolidated=False)

    # Just focus on GGA data. This prevents data from multiple sources,
    # which can cause duplicate time stamps, or even reversing time sequences
    ed_ds = ed_ds.where((ed_ds["sentence_type"]=="GGA"), drop=True)

    # Add the latitude and longitudes to our list of DataArrays
    latitudes.append(ed_ds.latitude)
    longitudes.append(ed_ds.longitude)

# Concatenate them all together:
all_latitudes_da = xr.concat(latitudes, dim='time1')
all_longitudes_da = xr.concat(longitudes, dim='time1')
all_latitudes_da

CPU times: user 401 ms, sys: 40.9 ms, total: 442 ms
Wall time: 406 ms


<xarray.DataArray 'latitude' (time1: 582)> Size: 5kB
array([27.28920035, 27.2892065 , 27.2892202 , 27.28922928, 27.2892373 ,
       27.28924642, 27.2892554 , 27.28926188, 27.28926773, 27.28927468,
       27.28928022, 27.28928235, 27.28928055, 27.28928157, 27.28928272,
       27.28929077, 27.28931223, 27.28933043, 27.28933738, 27.28933915,
       27.28935617, 27.28936635, 27.28938275, 27.28943607, 27.28944378,
       27.28943315, 27.28945997, 27.28945097, 27.28949758, 27.2895192 ,
       27.28950042, 27.28952308, 27.2895237 , 27.2895307 , 27.2895347 ,
       27.28954055, 27.28953968, 27.28953673, 27.28954393, 27.28954935,
       27.28955633, 27.28956283, 27.28956905, 27.2895809 , 27.28959148,
       27.2896009 , 27.28960662, 27.28961053, 27.28961638, 27.28962647,
       27.28963803, 27.28964835, 27.2896571 , 27.28966963, 27.2896828 ,
       27.2896947 , 27.28970713, 27.28971758, 27.28972808, 27.28974127,
       27.28975728, 27.28977667, 27.28979218, 27.28980208, 27.28981247,
       27.28982482, 27.2898346 , 27.28984523, 27.28985695, 27.28986895,
       27.28987817, 27.28988488, 27.2898975 , 27.2899098 , 27.28991915,
       27.28992757, 27.28993817, 27.28994868, 27.28995627, 27.28996213,
       27.28996985, 27.28997783, 27.28998375, 27.28998742, 27.28999203,
       27.28999443, 27.28999652, 27.29000453, 27.29001907, 27.2900303 ,
       27.29004138, 27.29005167, 27.29005878, 27.2900693 , 27.29008063,
       27.29009065, 27.29010472, 27.29011593, 27.29013355, 27.2901439 ,
...
       27.29409812, 27.29410672, 27.29411552, 27.29412527, 27.29413608,
       27.2941453 , 27.29415698, 27.29416952, 27.29418233, 27.29419363,
       27.29420337, 27.29421165, 27.29422058, 27.2942297 , 27.29423943,
       27.29425303, 27.29426768, 27.29428032, 27.29429762, 27.29430783,
       27.29431883, 27.2943303 , 27.29434032, 27.29435433, 27.29436543,
       27.29437503, 27.29438505, 27.2943964 , 27.29441117, 27.29442632,
       27.29444105, 27.29445363, 27.29446487, 27.29447353, 27.29447918,
       27.29448525, 27.2944909 , 27.29449582, 27.29450118, 27.29450743,
       27.29451378, 27.29451997, 27.29452608, 27.29453108, 27.29453928,
       27.29454587, 27.29455475, 27.29456552, 27.29457798, 27.29459122,
       27.29460387, 27.29461518, 27.29462553, 27.29463482, 27.29464348,
       27.29465435, 27.29466678, 27.29468287, 27.29469475, 27.29470552,
       27.29471042, 27.2947182 , 27.29473038, 27.29474472, 27.29476258,
       27.29477827, 27.29478877, 27.29479468, 27.29479888, 27.29480168,
       27.29480655, 27.2948115 , 27.29481927, 27.2948233 , 27.29482433,
       27.29481852, 27.29481567, 27.29482098, 27.29483412, 27.29484798,
       27.29486225, 27.29487585, 27.29488983, 27.29490243, 27.29491298,
       27.29492332, 27.29493517, 27.29494768, 27.29495955, 27.29496893,
       27.29497897, 27.29499013, 27.29500138, 27.2950091 , 27.29501463,
       27.29500767, 27.29499528])
Coordinates:
  * time1    (time1) datetime64[ns] 5kB 2025-04-30T22:11:10.096977 ... 2025-0...
Attributes:
    long_name:      Platform latitude
    standard_name:  latitude
    units:          degrees_north
    valid_range:    (-90.0, 90.0)

In [7]:
# Look for and fix reversed time values:
if echopype.qc.exist_reversed_time(all_latitudes_da, "time1"):
    # Coerce increasing time
    echopype.qc.coerce_increasing_time(all_latitudes_da)

KeyError: 'ping_time'

In [240]:
all_latitudes_da.time1.values

array(['2025-04-30T22:11:10.096977000', '2025-04-30T22:11:11.743734000',
       '2025-04-30T22:11:12.741459000', '2025-04-30T22:11:13.744152000',
       '2025-04-30T22:11:14.746864000', '2025-04-30T22:11:15.743572000',
       '2025-04-30T22:11:16.743282000', '2025-04-30T22:11:17.742008000',
       '2025-04-30T22:11:18.742701000', '2025-04-30T22:11:19.746414000',
       '2025-04-30T22:11:20.744139000', '2025-04-30T22:11:21.744848000',
       '2025-04-30T22:11:22.743542000', '2025-04-30T22:11:23.743250000',
       '2025-04-30T22:11:24.746964000', '2025-04-30T22:11:25.747673000',
       '2025-04-30T22:11:26.744381000', '2025-04-30T22:11:27.743090000',
       '2025-04-30T22:11:28.747803000', '2025-04-30T22:11:29.747513000',
       '2025-04-30T22:11:30.743219000', '2025-04-30T22:11:31.743929000',
       '2025-04-30T22:11:32.743637000', '2025-04-30T22:11:33.744349000',
       '2025-04-30T22:11:34.747063000', '2025-04-30T22:11:35.745787000',
       '2025-04-30T22:11:36.743477000', '2025-04-30

In [244]:
time_diffs = np.diff(all_latitudes_da.time1.values)
problem_indices = np.where(time_diffs <= 0)[0]
problem_indices

array([509])

In [245]:
all_latitudes_da.time1.isel(time1=slice(500, 520))

<xarray.DataArray 'time1' (time1: 20)> Size: 160B
array(['2025-04-30T22:19:30.754904000', '2025-04-30T22:19:31.755613000',
       '2025-04-30T22:19:32.755323000', '2025-04-30T22:19:33.755033000',
       '2025-04-30T22:19:34.766752000', '2025-04-30T22:19:35.757456000',
       '2025-04-30T22:19:36.755163000', '2025-04-30T22:19:37.754871000',
       '2025-04-30T22:19:38.753580000', '2025-04-30T22:19:39.760296000',
       '2025-04-30T22:19:39.756693000', '2025-04-30T22:19:41.753710000',
       '2025-04-30T22:19:42.752419000', '2025-04-30T22:19:43.755131000',
       '2025-04-30T22:19:44.758844000', '2025-04-30T22:19:45.755552000',
       '2025-04-30T22:19:46.756260000', '2025-04-30T22:19:47.754971000',
       '2025-04-30T22:19:48.756681000', '2025-04-30T22:19:49.759394000'],
      dtype='datetime64[ns]')
Coordinates:
  * time1    (time1) datetime64[ns] 160B 2025-04-30T22:19:30.754904 ... 2025-...
Attributes:
    axis:           T
    comment:        Time coordinate corresponding to NMEA position data.
    long_name:      Timestamps for NMEA datagrams
    standard_name:  time

In [242]:
# Subsample to 10 second boundaries:
all_latitudes_resampled = all_latitudes_da.resample(time1='10s').interpolate('linear')

all_latitudes_resampled

ValueError: Index must be monotonic for resampling